# Overview
- lightGBMのハイパラチューニング

# Const

In [1]:
NB = '009'
N_SPLITS = 5
SHOW_LOG = False


PATH_TRAIN = './../data/official/train.csv'
PATH_TEST = './../data/official/test.csv'
PATH_SAMPLE_SUBMITTION = './../data/official/atmaCup8_sample-submission.csv'
SAVE_DIR = f'../data/output_nb/nb{NB}/'

feat_train_only = ['JP_Sales', 'Global_Sales', 'NA_Sales', 'Other_Sales', 'EU_Sales']
feat_common = ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher',
           'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer',
           'Rating']
feat_string = ['Platform', 'Genre', 'Publisher', 'Developer', 'Rating']

# Import everything I need :)

In [2]:
import os
import time
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor 

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')

# My function

In [3]:
def metric(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred) ** .5

def preprocess_User_Score(df):
    '''
    - tbdをnanにする
    - stringをfloatにする
    '''
    mask = df.User_Score.values == 'tbd'
    df.User_Score[mask] = np.nan
    df.User_Score = df.User_Score.values.astype(float)
    return df

def string_encode(df, cols):
    '''
    - np.nanがあれば、'nan'に置き換える
    - label encodingする
    '''
    df[cols] = df[cols].replace(np.nan, 'nan')
    for col in cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    return df

def df_preprocessing(df, string_cols):
    df = preprocess_User_Score(df)
    df = string_encode(df, string_cols)
    return df

In [14]:
def run_fold_lgbm(_X_trn, _y_trn, _X_val, _y_val, _X_te, model_config, show_log=True):
    FEAT_STRING = ['Platform', 'Genre', 'Publisher', 'Developer', 'Rating']
    
    # train
    model = LGBMRegressor(**model_config)
    model.fit(_X_trn, _y_trn.values[:, 0], categorical_feature=FEAT_STRING)
    
    # predict
    y_trn_pred = model.predict(_X_trn)
    y_val_pred = model.predict(_X_val)
    _y_test_pred = model.predict(_X_te)
    
    # postprocessiing
    y_trn_pred[y_trn_pred <= 1] = 1
    y_val_pred[y_val_pred <= 1] = 1
    
    if show_log:
        print(show_log)
        print(f'score train: {metric(_y_trn, y_trn_pred):.5f}')
        print(f'score valid: {metric(_y_val, y_val_pred):.5f}')
        print('')
    
    return y_trn_pred, y_val_pred, _y_test_pred

def run(X, y, X_te, splitter, use_col, model_config, show_log=True):
    print(f'use_col: {use_col}\n') if show_log else None

    oof = np.zeros(len(X))
    y_test_pred = np.zeros(len(X_te))
    for fold_i, (idx_trn, idx_val) in enumerate(splitter.split(X)):
        if show_log:
            print(f'::Fold {fold_i+1}/{N_SPLITS} start at {time.ctime()}::')
        X_trn, X_val = X.iloc[idx_trn, :], X.iloc[idx_val, :]
        y_trn, y_val = y.iloc[idx_trn], y.iloc[idx_val]
        X_trn = pd.DataFrame(X_trn, columns=X.columns)
        X_val = pd.DataFrame(X_val, columns=X.columns)

    #     # train
        y_trn_pred, y_val_pred, _y_test_pred = run_fold_lgbm(X_trn, y_trn, X_val, y_val, 
                                                             X_te, model_config, show_log=show_log)

        # result
        oof[idx_val] = y_val_pred
        y_test_pred += _y_test_pred / N_SPLITS


    y_test_pred[y_test_pred <= 1] = 1
    return oof, y_test_pred

# Preparation

set

In [5]:
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

<br>

load dataset

In [6]:
train = pd.read_csv(PATH_TRAIN)
test = pd.read_csv(PATH_TEST)
ss = pd.read_csv(PATH_SAMPLE_SUBMITTION)

<br>

preprocess

In [7]:
train = df_preprocessing(train, feat_string)
test = df_preprocessing(test, feat_string)

In [8]:
mask = test.columns != 'Name'
use_col = test.columns[mask].tolist()
use_col

['Platform',
 'Year_of_Release',
 'Genre',
 'Publisher',
 'Critic_Score',
 'Critic_Count',
 'User_Score',
 'User_Count',
 'Developer',
 'Rating']

In [9]:
X = train[use_col].copy()
y = train[['Global_Sales']].copy()
X_te = test[use_col].copy()

# Create Model

# Tuning

In [10]:
splitter = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2020)

In [11]:
model_config = {
    'random_state': 2020,
    'colsample_bytree': 0.7,
}

In [12]:
%%time
oof, y_test_pred = run(X, y, X_te, splitter, use_col, model_config, show_log=SHOW_LOG)
print(f'oof score: {metric(y, oof):.5f}')

oof score: 1.14436
CPU times: user 5.74 s, sys: 17.2 ms, total: 5.75 s
Wall time: 493 ms


In [ ]:
hyper_space = {'objective': 'regression',
               'metric':'mae',
               'boosting':'gbdt',
               'max_depth':  hp.choice('max_depth', [5, 8, 10, 15]),
               'num_leaves': hp.choice('num_leaves', [50, 100, 250]),
               'subsample': hp.choice('subsample', [.3, .5, .7, 1]),
               'subsample_freq': 1,
               'colsample_bytree': hp.choice('colsample_bytree', [ .6, .8, 1]),
               'learning_rate': hp.choice('learning_rate', [0.01, .1, .3]),
               'reg_alpha': hp.choice('reg_alpha', [.1, .2, .4, .6]),
               'reg_lambda':  hp.choice('reg_lambda', [.1, .2, .4, .6]),               
               'min_child_samples': hp.choice('min_child_samples', [20, 45, 70, 100]),
               'verbosity': -1,
               'bagging_seed': 11,
              }

In [22]:
# 最小化したい関数
def objective(trial):
    print('')
    params = {
        'n_estimators': int(trial.suggest_int('n_estimators', 10, 800)),
        'max_depth': trial.suggest_int('max_depth', 5, 300),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.1, 1.0, 0.1),
        'learning_rate': trial.suggest_loguniform('learing_rate', 1e-3, 1e-1),
        'reg_alpha': trial.suggest_discrete_uniform('reg_alpha', 1, 6, 1),
        'reg_lambda': trial.suggest_discrete_uniform('reg_alpha', 1, 6, 1),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
    }
    oof, y_test_pred = run(X, y, X_te, splitter, use_col, params, show_log=SHOW_LOG)
    return metric(y, oof)

In [ ]:
%%time
study = optuna.create_study()
study.optimize(objective, n_trials=10000)
print(study.best_trial)
print(f'\n best_params = {study.best_params}')

[I 2020-12-04 22:39:04,238] Finished trial#0 with value: 1.3130807923262957 with parameters: {'n_estimators': 606, 'max_depth': 218, 'subsample': 0.8, 'colsample_bytree': 0.6, 'learing_rate': 0.07152738622558157, 'reg_alpha': 1.0, 'min_child_samples': 98}. Best is trial#0 with value: 1.3130807923262957.


[I 2020-12-04 22:39:06,624] Finished trial#1 with value: 1.4066813732065426 with parameters: {'n_estimators': 524, 'max_depth': 24, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.7000000000000001, 'learing_rate': 0.002324221055558453, 'reg_alpha': 4.0, 'min_child_samples': 54}. Best is trial#0 with value: 1.3130807923262957.


[I 2020-12-04 22:39:08,507] Finished trial#2 with value: 1.268810195169164 with parameters: {'n_estimators': 412, 'max_depth': 189, 'subsample': 1.0, 'colsample_bytree': 0.6, 'learing_rate': 0.08319918228389797, 'reg_alpha': 4.0, 'min_child_samples': 36}. Best is trial#2 with value: 1.268810195169164.


[I 2020-12-04 22:39:10,441] Finished trial#3 with value: 1.457736972794275 with parameters: {'n_estimators': 492, 'max_depth': 255, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.30000000000000004, 'learing_rate': 0.0028974540983993788, 'reg_alpha': 3.0, 'min_child_samples': 30}. Best is trial#2 with value: 1.268810195169164.


[I 2020-12-04 22:39:14,227] Finished trial#4 with value: 1.2854259198650588 with parameters: {'n_estimators': 778, 'max_depth': 71, 'subsample': 0.5, 'colsample_bytree': 0.6, 'learing_rate': 0.046818239343290075, 'reg_alpha': 1.0, 'min_child_samples': 88}. Best is trial#2 with value: 1.268810195169164.


...

[I 2020-12-04 22:51:47,401] Finished trial#635 with value: 1.1273640932724616 with parameters: {'n_estimators': 347, 'max_depth': 139, 'subsample': 0.9, 'colsample_bytree': 0.9, 'learing_rate': 0.04022679345688777, 'reg_alpha': 6.0, 'min_child_samples': 21}. Best is trial#615 with value: 1.085890619618491.


[I 2020-12-04 22:51:48,619] Finished trial#636 with value: 1.1372557408856045 with parameters: {'n_estimators': 249, 'max_depth': 150, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.8, 'learing_rate': 0.06102058519776137, 'reg_alpha': 6.0, 'min_child_samples': 19}. Best is trial#615 with value: 1.085890619618491.


[I 2020-12-04 22:51:49,474] Finished trial#637 with value: 1.241834817480049 with parameters: {'n_estimators': 213, 'max_depth': 165, 'subsample': 0.1, 'colsample_bytree': 0.1, 'learing_rate': 0.022996323162469894, 'reg_alpha': 6.0, 'min_child_samples': 17}. Best is trial#615 with value: 1.085890619618491.


[I 2020-12-04 22:51:50,684] Finished trial#638 with value: 1.0863630813099001 with parameters: {'n_estimators': 234, 'max_depth': 179, 'subsample': 0.8, 'colsample_bytree': 0.9, 'learing_rate': 0.030146400830420486, 'reg_alpha': 6.0, 'min_child_samples': 16}. Best is trial#615 with value: 1.085890619618491.


[I 2020-12-04 22:51:51,544] Finished trial#639 with value: 1.1068894497827158 with parameters: {'n_estimators': 163, 'max_depth': 178, 'subsample': 0.8, 'colsample_bytree': 0.9, 'learing_rate': 0.02698424615345288, 'reg_alpha': 6.0, 'min_child_samples': 16}. Best is trial#615 with value: 1.085890619618491.
